# Projeto 1 - Ciência dos Dados

Nome:GRASIELLY ALMEIDA LIMA

Nome:QUÉZIA PEREIRA DA SILVA MOURA

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import nltk 
from nltk.stem import RSLPStemmer
from sklearn.model_selection import train_test_split
nltk.download('stopwords')

pd.options.mode.chained_assignment = None  # default='warn'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\quezi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\quezi\OneDrive\Área de Trabalho\P1_CDados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'ROCK IN RIO.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,CLASSIFICAÇÃO
0,"aiai, rock in rio q me aguardeeee https://t.co...",1
1,quero uma apresentação dessa no rock in rio ...,1
2,ainda procuro gente pra ir no rock in rio 😢,0
3,"o rock in rio me respondendo, socorro https://...",0
4,alguém trás a rihanna pro rock in rio novamente,1


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,CLASSIFICAÇÃO
0,será que o justin vai fazer um show no rj além...,1
1,isso aqui no rock in rio acho que vou chorar ...,1
2,nao to preparada para tudo isso no rock in rio...,1
3,@acciomari quanto ta o ingresso do rock in rio?,1
4,meu namorado vai realizar meu sonho de ir pro ...,0


___
## Classificador automático de sentimento


O produto escolhido foi o Rock in Rio,"O Rock in Rio é um festival de música idealizado pelo empresário brasileiro Roberto Medina pela primeira vez em 1985, sendo, desde sua criação, reconhecidamente, o maior festival musical do planeta." Em especial foi escolhido o evento que ocorrerá em 2022 na cidade do Rio de Janeiro.

FONTE: https://pt.wikipedia.org/wiki/Rock_in_Rio

Quanto o critério de classificação dos tweets, classificamos como tweets relevantes toda e qualquer citação, positiva ou negativa, referente às citações dos internautas acerca do produto. Tweets considerados como "irrelevantes" foram denominados como tweets não relacionados diretamente ao produto. Além disso, os tweets considerados relevantes expressavam algum sentimento em relação ao evento musical, seja de felicidade, ansiedade, informações sobre compra de ingressos e possíveis atrações que estarão presentes no evento de 2022.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

O algoritmo "Naive Bayes" é um classificador probabilístico muito utilizado em aprendizado de máquina, que se baseia no "Teorema de Bayes". Hoje também é utilizado na área de Aprendizado de Máquina para categorizar textos com base na frequência das palavras usadas.

Ele recebe o nome de "naive" (ingênuo) porque desconsidera a correlação entre as variáveis (características).

Frequentemente aplicado em processamento de linguagem natural e diagnósticos médicos, o método pode ser usado quando os atributos que descrevem as instâncias for condicionalmente independentes. Ou seja, o teorema de Bayes trata sobre probabilidade condicional. Isto é, qual a probabilidade de o evento A ocorrer, dado o evento B.

Desse modo, a resolução de problemas relacionados a texto é muito bem resolvida com a utilização do Naive Bayes. Classificação de textos, filtragem de SPAM e análise de sentimento em redes sociais são algumas das muitas aplicações para o algoritmo.

FONTE: https://www.datageeks.com.br/naive-bayes/

**EVENTOS**

 * $R$: tweets relevantes
 * $R^c$: tweets irrelevantes
 * $C$: tweet recebido

**PROBABILIDADES**

 * $P(R)$ : probabilidade de um tweet ser relevante ;
 * $P(R^c)$: probabilidade de um tweet não ser relevante, ou seja, irrelevante ;
 * $P(C)$ : probabilidade de cada tweet ocorrer na língua portuguesa ;
 * $P(C|R)$ : probabilidade do tweet existir dado o conjunto de tweets relevantes ;
 * $P(C|R^c)$: probabilidade do tweet existir dado o conjunto de tweets irrelevantes ; 
 * $P(R|C)$: probabilidade do tweet ser relevante dado a frase ;
 * $P(R^c|C)$: probabilidade do tweet ser irrelevante dado a frase;


$$P(R|C) = \frac{P(C|R) \cdot P(R)}{P(C)}$$

### Funções criadas:

In [6]:
import re 
from nltk.tokenize.casual import TweetTokenizer #Para separar os emojis como palavras.


# Removendo caracteres e outras limpezas 

def cleanup(texto):
    pontuacao = '[.,@"''"-)(][):;\/?!]'
    modelo = re.compile(pontuacao)
    novo_texto = re.sub(modelo,'',texto)
    novo_texto = re.sub(r'http\S+','', novo_texto)
    tt = TweetTokenizer()
    novo_texto = tt.tokenize(novo_texto)
    return novo_texto

# Função para remover espaços

def Remove_espaco(lista_palavras):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    nova_lista_palavras = []
    for palavra in lista_palavras:
        if palavra not in stopwords:
            nova_lista_palavras.append(palavra)
    return nova_lista_palavras

def Suavizacao_Laplace(a, b, c):
    x = (1 + a)/(b + len(c))  
    return x

# Função para multiplicar valores de listas
def Multiplica_valores(lista) : 
    resultado = 1
    for elemento in lista : 
         resultado = resultado * elemento 
    return resultado

    



### Criando variáveis com palavras juntas e as limpando:

In [7]:
# TWEETS RELEVANTES DO TREINAMENTO

train_relevante = train.loc[train.CLASSIFICAÇÃO == 1, :]
train_filtrado_relevante = " ".join(train_relevante['Treinamento']) 

# TWEETS IRRELEVANTES DO TREINAMENTO
train_irrelevante = train.loc[train.CLASSIFICAÇÃO == 0, :]
train_filtrado_irrelevante = " ".join(train_irrelevante['Treinamento']) 
train_irrelevante.Treinamento

TypeError: sequence item 127: expected str instance, float found

In [ ]:
#LIMPANDO VARIÁVEIS CRIADAS, COLOCANDO EM MINÚSCULO E AS GUARDANDO
relevante = cleanup(train_filtrado_relevante.lower()) # Colocar todas letras em minúsculo e Limpar variáveis
palavras_relevantes = Remove_espaco(relevante)  

irrelevante = cleanup(train_filtrado_irrelevante.lower())
palavras_irrelevantes = Remove_espaco(irrelevante)

In [ ]:
total_palavras = palavras_relevantes + palavras_irrelevantes

### Cálculo das frequências

### Atribuindo funções ao classificador

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**